In [ ]:
import os
import mlflow
import pandas as pd
import pickle
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from pathlib import Path
from datetime import datetime
from sklearn.metrics import root_mean_squared_error
from datetime import datetime


### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [21]:
#MLFLOW_TRACKING_URI = "sqlite:///../mlflow_db/mlflow.db"
mlflow_df_path = "/home/ubuntu/mlops-dlp/mlflow_db/mlflow.db"
MLFLOW_TRACKING_URI = f"sqlite:///{mlflow_df_path}"

In [22]:
print(os.getcwd())
if os.path.exists("../mlflow_db/mlflow.db"):
    print("MLflow database already exists.")

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

/home/ubuntu/mlops-dlp/week02
MLflow database already exists.


In [23]:
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops-dlp/week02/mlruns/5', creation_time=1751424398759, experiment_id='5', last_update_time=1751424398759, lifecycle_stage='active', name='my-cool-experiment_dlp', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops-dlp/week02/mlruns/4', creation_time=1751399305164, experiment_id='4', last_update_time=1751399305164, lifecycle_stage='active', name='my-cool-experiment__', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops-dlp/mlruns/1', creation_time=1751320603220, experiment_id='1', last_update_time=1751320603220, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [24]:
experiment_data = client.search_experiments(
    filter_string="name = 'my-cool-experiment_dlp'",
    # view_type=ViewType.ACTIVE_ONLY,
)
experiment_exists = len(experiment_data) > 0
experiment_exists

True

In [25]:
if not experiment_exists:
    print("Experiment does not exist, creating it.")
    client.create_experiment(
        name="my-cool-experiment_dlp"
        #artifact_location=experiment_path,
    )

In [26]:
runs_xgb = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.valid_rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.valid_rmse ASC"]
)

In [27]:
runs_sklearn = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [29]:
for run in runs_xgb:
    print(f"xgb run id: {run.info.run_id}, rmse: {run.data.metrics['valid_rmse']:.4f}")
for run in runs_sklearn:
    print(f"sklearn run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

xgb run id: c63debf686624373847fb76d27d70dea, rmse: 6.5131
xgb run id: 2ff312aa59b8489bb5bdb2434a60358e, rmse: 6.5131
sklearn run id: 050727b8b6c54343a9a4cdf006f31f31, rmse: 6.7423
sklearn run id: a274a4c8b8104c61bd4da1ce6bfebb1a, rmse: 6.9091
sklearn run id: 7d36487eb3514a3fb71c69ff27c95158, rmse: 6.9334


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [33]:
run_id = "c63debf686624373847fb76d27d70dea"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1751425192037, current_stage='None', description=None, last_updated_timestamp=1751425192037, name='nyc-taxi-regressor', run_id='c63debf686624373847fb76d27d70dea', run_link=None, source='/home/ubuntu/mlops-dlp/mlruns/1/c63debf686624373847fb76d27d70dea/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [46]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.aliases}, run_id: {version.run_id}")

version: 5, stage: ['Staging'], run_id: c63debf686624373847fb76d27d70dea


/tmp/ipykernel_224702/1353522843.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [84]:
model_name = "nyc-taxi-regressor"

model_version = 1
new_alias = "Production"
client.set_registered_model_alias(name=model_name, alias=new_alias, version=f"{model_version}")
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}"
)

new_alias = "Staging"
for model_version in range(2, 6):
    print(f"Setting model version {model_version} to alias {new_alias}")
    client.set_registered_model_alias(name=model_name, alias=new_alias, version=f"{model_version}")
    client.update_model_version(
        name=model_name,
        version=model_version,
        description=f"The model version {model_version} was transitioned to {new_alias} on {date}"
    )


Setting model version 2 to alias Staging
Setting model version 3 to alias Staging
Setting model version 4 to alias Staging
Setting model version 5 to alias Staging


### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [80]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    #df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, model_alias, X_test, y_test):
    model_uri = f"models:/{model_name}@{model_alias}"
    model = mlflow.pyfunc.load_model(model_uri)
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [70]:
test_data_path = "~/Data/green_tripdata_2021-03.parquet"
df = read_dataframe(test_data_path)

In [71]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/ubuntu/mlops-dlp/week02/preprocessor'

In [73]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [74]:
X_test = preprocess(df, dv)
target = "duration"
y_test = df[target].values

In [88]:
model_name = "nyc-taxi-regressor"
model_alias = "testing"

# Get the model version using a model URI
#model_uri = f"models:/{model_name}@{model_version_alias}"
%time test_model(name=model_name, model_alias=model_alias, X_test=X_test, y_test=y_test)

OSError: No such file or directory: '/home/ubuntu/mlops-dlp/mlruns/1/c63debf686624373847fb76d27d70dea/artifacts/model/.'

In [87]:
print(model)

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: c63debf686624373847fb76d27d70dea

